### Extract Transform Load (ETL)

ETL is one of the first things which needs to be done in a data science project. The nature of this task highly depends on the type of data source. Whether it is relational or unstructured, enterprise data or internet data, persistent data or streaming data. This heavily influences the choice of architecture. Therefore, you must document your choice and thinking process in the Architectural Decision Document (ADD).

This task involves – as the name implies – accessing the data source, transforming it in a way it can be easily worked with and finally make it available to downstream analytics processes – either real-time streaming or batch ones.

In case of operational relational data, de-normalization usually needs to take place, for unstructured data, some feature extraction might already be appropriate and for real-time data, windows are usually created.

Please create an ETL process, document it and save this deliverable according to the naming convention of the process model.

In [3]:
# install backend tensorflow
!pip install tensorflow
!pip install python-mnist
!pip install Pillow


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 86.3 MB 109 kB/s  eta 0:00:01█████▉        | 64.4 MB 63.8 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 63.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 39.8 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.0
  

In [4]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt
from mnist import MNIST
from PIL import Image

In [5]:
tf.__version__

'2.0.0'

In [6]:
PIL.__version__

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [8]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz?raw=True
!mv train-images-idx3-ubyte.gz?raw=True train-images-idx3-ubyte.gz
!gunzip train-images-idx3-ubyte.gz
!ls -lahr train-images-idx3-ubyte

--2021-05-25 17:36:08--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18165135 (17M)
Saving to: ‘train-images-idx3-ubyte.gz?raw=True’

train-images-idx3-u 100%[===================>]  17.32M  6.76MB/s    in 2.6s    

2021-05-25 17:36:11 (6.76 MB/s) - ‘train-images-idx3-ubyte.gz?raw=True’ saved [18165135/18165135]

-rw-rw---- 1 wsuser watsonstudio 45M Feb  4  2019 train-images-idx3-ubyte


In [9]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz?raw=True
!mv train-labels-idx1-ubyte.gz?raw=True train-labels-idx1-ubyte.gz
!gunzip train-labels-idx1-ubyte.gz
!ls -lahr train-labels-idx1-ubyte

--2021-05-25 17:36:42--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29497 (29K)
Saving to: ‘train-labels-idx1-ubyte.gz?raw=True’

train-labels-idx1-u 100%[===================>]  28.81K  --.-KB/s    in 0.1s    

2021-05-25 17:36:43 (211 KB/s) - ‘train-labels-idx1-ubyte.gz?raw=True’ saved [29497/29497]

-rw-rw---- 1 wsuser watsonstudio 59K Feb  4  2019 train-labels-idx1-ubyte


In [10]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-images-idx3-ubyte.gz?raw=True
!mv t10k-images-idx3-ubyte.gz?raw=True t10k-images-idx3-ubyte.gz
!gunzip t10k-images-idx3-ubyte.gz
!ls -lahr t10k-images-idx3-ubyte

--2021-05-25 17:37:12--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-images-idx3-ubyte.gz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3041136 (2.9M)
Saving to: ‘t10k-images-idx3-ubyte.gz?raw=True’

t10k-images-idx3-ub 100%[===================>]   2.90M  2.83MB/s    in 1.0s    

2021-05-25 17:37:14 (2.83 MB/s) - ‘t10k-images-idx3-ubyte.gz?raw=True’ saved [3041136/3041136]

-rw-rw---- 1 wsuser watsonstudio 7.5M Feb  4  2019 t10k-images-idx3-ubyte


In [11]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-labels-idx1-ubyte.gz?raw=True
!mv t10k-labels-idx1-ubyte.gz?raw=True t10k-labels-idx1-ubyte.gz
!gunzip t10k-labels-idx1-ubyte.gz
!ls -lahr t10k-labels-idx1-ubyte

--2021-05-25 17:37:30--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-labels-idx1-ubyte.gz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5120 (5.0K)
Saving to: ‘t10k-labels-idx1-ubyte.gz?raw=True’

t10k-labels-idx1-ub 100%[===================>]   5.00K  --.-KB/s    in 0s      

2021-05-25 17:37:31 (274 MB/s) - ‘t10k-labels-idx1-ubyte.gz?raw=True’ saved [5120/5120]

-rw-rw---- 1 wsuser watsonstudio 9.8K Feb  4  2019 t10k-labels-idx1-ubyte


In [12]:
url = "http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist_classmap.csv"
df_classmap = pd.read_csv(url)
phonetic = ['o','ki','su','tsu','na','ha','ma','ya','re','wo']
df_classmap['phonetic'] = phonetic
df_classmap

,index,codepoint,char,phonetic
0,0,U+304A,お,o
1,1,U+304D,き,ki
2,2,U+3059,す,su
3,3,U+3064,つ,tsu
4,4,U+306A,な,na
5,5,U+306F,は,ha
6,6,U+307E,ま,ma
7,7,U+3084,や,ya
8,8,U+308C,れ,re
9,9,U+3092,を,wo


In [14]:
!mkdir kmnistdata
!cp t10k-images-idx3-ubyte kmnistdata/t10k-images-idx3-ubyte
!cp t10k-labels-idx1-ubyte kmnistdata/t10k-labels-idx1-ubyte
!cp train-images-idx3-ubyte kmnistdata/train-images-idx3-ubyte
!cp train-labels-idx1-ubyte kmnistdata/train-labels-idx1-ubyte
!ls -al kmnistdata

mkdir: cannot create directory ‘kmnistdata’: File exists
total 53680
drwxrwx--- 2 wsuser watsonstudio     4096 May 25 17:40 .
drwxr-x--- 3 wsuser watsonstudio     4096 May 25 17:39 ..
-rw-rw---- 1 wsuser watsonstudio  7840016 May 25 17:40 t10k-images-idx3-ubyte
-rw-rw---- 1 wsuser watsonstudio    10008 May 25 17:40 t10k-labels-idx1-ubyte
-rw-rw---- 1 wsuser watsonstudio 47040016 May 25 17:40 train-images-idx3-ubyte
-rw-rw---- 1 wsuser watsonstudio    60008 May 25 17:40 train-labels-idx1-ubyte


In [15]:
data = MNIST('kmnistdata')
train_images, train_labels = data.load_training()
test_images, test_labels = data.load_testing()

In [16]:

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [17]:
train_images = train_images / 255
test_images = test_images / 255

In [18]:
train_images[5]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.05490196, 0.27058824, 0.00392157,
       0.07843137, 0.39607843, 0.30196078, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00392157, 0.02745098,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.54901961, 0.97254902, 0.11764706, 0.00392157, 0.14117647,
       0.00784314, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.18431373, 0.66666667, 0.40784314, 0.20392157,
       0.07058824, 0.03529412, 0.01568627, 0.00784314, 0.        ,
       0.        , 0.00784314, 0.03137255, 0.78431373, 1.     